In [1]:
import pandas as pd
from h5py import File
from pathlib import Path
import json
import tqdm
import transformers as trf
import os
os.environ['USER'] = "rmoine"
import llama.main as m

c:\Users\robin\miniconda3\envs\severityPrediction\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'
Import of import evaluate failed


In [4]:
root_folder = Path("../../data/embeddings")
dataset = "mozilla"
ref_path = root_folder.parent / f"llm" 
ref_path = list(ref_path.rglob(f"{dataset}_*.json"))[0]
hdf5_path = root_folder / f"embeddings_{ref_path.stem}.hdf5"
print(hdf5_path.resolve())
print(ref_path.resolve())
assert hdf5_path.exists()

C:\Users\robin\Documents\projets\severityPrediction\data\embeddings\embeddings_mozilla_200k.hdf5
C:\Users\robin\Documents\projets\severityPrediction\data\llm\mozilla_200k.json


In [5]:
bug_id_hdf5 = set({})
with File(hdf5_path) as fp:
    for k in fp:
        bug_id_hdf5.add(int(k))

In [6]:
bug_id_ref = set({})
with open(ref_path) as fp:
    data_ref = json.load(fp)
    for e in data_ref:
        bug_id_ref.add(e['bug_id'])

In [8]:
print(f"{len(bug_id_hdf5)=}\\n{len(bug_id_ref)=}\n{len(bug_id_ref)-len(bug_id_hdf5)=}\n{len(bug_id_ref.difference(bug_id_hdf5))=}")

len(bug_id_hdf5)=200723\nlen(bug_id_ref)=200824
len(bug_id_ref)-len(bug_id_hdf5)=101
len(bug_id_ref.difference(bug_id_hdf5))=101


In [9]:
tokenizer = m.get_tokenizer(token=m.default_token,model_name=m.default_model)
missing = []
for e in tqdm.tqdm(data_ref):
    if not (e['bug_id']) in bug_id_ref.difference(bug_id_hdf5):
        continue
    e['tokens_description'] = tokenizer.convert_ids_to_tokens(tokenizer(e['description'])['input_ids'])
    e['n_tokens_description'] = len(e['tokens_description'])
    missing.append(e)


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to C:\Users\robin\.cache\huggingface\token
Login successful


100%|██████████| 200824/200824 [22:42<00:00, 147.36it/s]


In [3]:
path = hdf5_path.parent / f"{hdf5_path.stem}_missing.json"
if path.exists():
    df = pd.read_json(path)
else:
    df = pd.DataFrame(missing)
    df.to_json(path,orient="records")

In [4]:
len(df),df.columns

(101,
 Index(['bug_id', 'bug_severity', 'binary_severity', 'description',
        'stemmed_description', 'tokens_description', 'n_tokens_description'],
       dtype='object'))

In [6]:
df

,bug_id,bug_severity,binary_severity,description,stemmed_description,tokens_description,n_tokens_description
0,255059,critical,1,close applet jFrame window crash browser,close applet jframe window crash browser,"[<s>, ▁close, ▁app, let, ▁j, Frame, ▁window, ▁...",9
1,255050,minor,0,saving draft progress dialog box title first s...,save draft progress dialog box titl first say ...,"[<s>, ▁saving, ▁draft, ▁progress, ▁dialog, ▁bo...",17
2,255063,minor,0,Installing Firefox 0.9.3 n't remove 0.9.2 unin...,instal firefox 0.9.3 n't remov 0.9.2 uninstal ...,"[<s>, ▁Install, ing, ▁Firefox, ▁, 0, ., 9, ., ...",24
3,255060,blocker,1,Firefox Thunderbird migration requires Seamonk...,firefox thunderbird migrat requir seamonkey cr...,"[<s>, ▁Firefox, ▁Th, under, bird, ▁migration, ...",16
4,255065,major,1,migration occurs Eudora Mac OS X,migrat occur eudora mac os x,"[<s>, ▁migration, ▁occurs, ▁E, ud, ora, ▁Mac, ...",9
...,...,...,...,...,...,...,...
96,255366,blocker,1,File picker calls crash mozilla open file file...,file picker call crash mozilla open file file ...,"[<s>, ▁File, ▁pick, er, ▁calls, ▁crash, ▁m, oz...",16
97,255373,minor,0,Selecting bookmark properties right clicking b...,select bookmark properti right click bookmark ...,"[<s>, ▁Select, ing, ▁book, mark, ▁properties, ...",12
98,255371,critical,1,Crash openning webpages nsTextFrame :TextStyle...,crash open webpag nstextfram :textstyl :textstyl,"[<s>, ▁Cr, ash, ▁open, ning, ▁web, pages, ▁ns,...",16
99,255379,minor,0,Firefox remembers two distinct download direct...,firefox rememb two distinct download directori,"[<s>, ▁Firefox, ▁rem, embers, ▁two, ▁distinct,...",8


In [5]:
df['n_tokens_description'].describe()

count    101.000000
mean      13.138614
std        5.669268
min        5.000000
25%        9.000000
50%       12.000000
75%       16.000000
max       38.000000
Name: n_tokens_description, dtype: float64

In [38]:
df.query("n_tokens_description < 300").iloc[0].tokens_description

['<s>',
 '▁close',
 '▁app',
 'let',
 '▁j',
 'Frame',
 '▁window',
 '▁crash',
 '▁browser']